In [ ]:
!pip install unsloth

In [ ]:
!pip install fastapi uvicorn pyngrok

In [ ]:
!pip install -U bitsandbytes

In [4]:
from fastapi import FastAPI
import torch
from unsloth import FastLanguageModel
import uvicorn
from datetime import datetime
from pyngrok import ngrok

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [ ]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "t8ix/lora_latoken_llama_new",
    max_seq_length = 2048,
    dtype = None,
    load_in_4bit = True,
)

FastLanguageModel.for_inference(model)

In [13]:
app = FastAPI()

model = model.to("cuda")

In [7]:
from getpass import getpass
ngrok_token = getpass('Enter your ngrok token: ')

!ngrok authtoken {ngrok_token}

Enter your ngrok token: ··········
Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
# Start ngrok tunnel
public_url = ngrok.connect(8000)
print(f"Public URL: {public_url}")

In [9]:
from pydantic import BaseModel
class Message(BaseModel):
    message: str

In [10]:
from unsloth.chat_templates import get_chat_template

# Assuming `tokenizer` is already defined
tokenizer = get_chat_template(
    tokenizer,
    mapping={"role": "from", "content": "value", "user": "human", "assistant": "gpt"},
    chat_template="chatml",
)

Unsloth: Will map <|im_end|> to EOS = <|end_of_text|>.


In [ ]:
messages = [
    {"from": "human", "value": "Хочу зарегистрироваться на хакатон? Где это можно сделать?"},
    ]


inputs = tokenizer.apply_chat_template(
    messages,
    tokenize=True,
    add_generation_prompt=True,
    return_tensors="pt",
).to("cuda")

response = model.generate(input_ids=inputs, max_new_tokens=248)

In [ ]:
tokenizer.decode(response[0], skip_special_tokens=True)

In [14]:
@app.post("/generate")
async def generate(payload: Message):

    messages = [
    {"from": "human", "value": payload.message},
    ]

    inputs = tokenizer.apply_chat_template(
    messages,
    tokenize=True,
    add_generation_prompt=True,
    return_tensors="pt",
    ).to("cuda")

    response = model.generate(input_ids=inputs, max_new_tokens=512)
    return {"response": tokenizer.decode(response[0], skip_special_tokens=True).split("<|im_start|>assistant\n")[-1]}

In [17]:
import nest_asyncio
nest_asyncio.apply()

if __name__ == "__main__":
    uvicorn.run(app, host="0.0.0.0", port=8000)

INFO:     Started server process [1324]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


INFO:     186.78.16.65:0 - "POST /generate HTTP/1.1" 200 OK
INFO:     186.78.16.65:0 - "POST /generate HTTP/1.1" 200 OK
INFO:     186.78.16.65:0 - "POST /generate HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [1324]
